In [1]:
## Imports
import pandas as pd
from sklearn import preprocessing
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2,RFE
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble._bagging import BaggingClassifier
from sklearn.ensemble._forest import ExtraTreesClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble._bagging import BaggingClassifier
from sklearn.metrics import confusion_matrix, fbeta_score

#!pip install fairlearn
from fairlearn.metrics import MetricFrame
from fairlearn.metrics import selection_rate, count
from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.reductions import ExponentiatedGradient, ClassificationMoment, Moment, DemographicParity, GridSearch
from fairlearn.metrics import equalized_odds_difference, equalized_odds_ratio, false_negative_rate_difference, false_positive_rate_difference, true_positive_rate_difference, true_negative_rate_difference, demographic_parity_ratio, demographic_parity_difference

# Suppress warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
%matplotlib inline

In [2]:
df = pd.read_csv('Teds4.csv')

In [3]:
pd.set_option('display.max_columns',1000)

In [4]:
df.head()

,LIVARAG_D,SUB1_D,OPSYNFLG,FREQ2_D,PSYPROB,ALCDRUG,PSOURCE,HLTHINS,DETNLF_D,MARFLG,CBSA2010,SUB2,ROUTE1,REASON,BENZFLG,PRIMPAY,EMPLOY_D,FRSTUSE1,SUB3,FREQ3,SUB1,DSMCRIT,FREQ1,LIVARAG,MTHAMFLG,METHUSE,NOPRIOR,DAYWAIT,EDUC,EMPLOY,REGION,AGE,STFIPS,DETCRIM,DIVISION,RACE,FREQ1_D,PREG,FREQ2,FREQ_ATND_SELF_HELP_D,PRIMINC,DETNLF,FREQ_ATND_SELF_HELP,VET,L/S
0,3,2,0,-9,1,1,1,3,-9,0,-9,1,1,1,0,7,2,5,1,-9,2,19,2,3,0,2,0,0,4,2,4,4,2,-9,9,1,2,2,-9,1,1,-9,1,2,1.0
1,3,2,0,2,1,3,1,3,-9,1,-9,4,1,3,0,7,1,3,1,-9,2,4,2,3,0,2,0,0,3,1,4,5,2,-9,9,1,2,2,3,1,1,-9,1,2,1.0
2,3,2,0,2,1,3,1,2,2,1,-9,4,1,3,0,4,4,2,1,-9,2,19,2,3,0,2,0,0,2,4,4,1,2,-9,9,8,2,2,2,1,4,2,1,2,1.0
3,3,2,0,3,1,3,1,3,-9,1,-9,5,1,1,0,7,3,3,4,1,2,4,1,3,0,2,1,0,3,3,4,5,2,-9,9,1,1,2,1,4,4,-9,4,2,1.0
4,3,2,0,-9,1,1,1,3,3,0,-9,1,1,3,0,7,4,7,1,-9,2,4,1,3,0,2,1,0,3,4,4,11,2,-9,9,8,1,2,-9,1,4,3,1,2,1.0


In [5]:
df.columns

Index(['LIVARAG_D', 'SUB1_D', 'OPSYNFLG', 'FREQ2_D', 'PSYPROB', 'ALCDRUG',
       'PSOURCE', 'HLTHINS', 'DETNLF_D', 'MARFLG', 'CBSA2010', 'SUB2',
       'ROUTE1', 'REASON', 'BENZFLG', 'PRIMPAY', 'EMPLOY_D', 'FRSTUSE1',
       'SUB3', 'FREQ3', 'SUB1', 'DSMCRIT', 'FREQ1', 'LIVARAG', 'MTHAMFLG',
       'METHUSE', 'NOPRIOR', 'DAYWAIT', 'EDUC', 'EMPLOY', 'REGION', 'AGE',
       'STFIPS', 'DETCRIM', 'DIVISION', 'RACE', 'FREQ1_D', 'PREG', 'FREQ2',
       'FREQ_ATND_SELF_HELP_D', 'PRIMINC', 'DETNLF', 'FREQ_ATND_SELF_HELP',
       'VET', 'L/S'],
      dtype='object')

In [6]:
X = df.drop(columns = ["L/S"])
y = df["L/S"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
df.shape

(1722503, 45)

In [8]:
models_to_train = [
    # LogisticRegression,
    RandomForestClassifier,
    # DecisionTreeClassifier,
    XGBClassifier,
    # BaggingClassifier
]

unfairness_metrics = [
    selection_rate,
    fbeta_score,
    equalized_odds_difference,
    equalized_odds_ratio
]
# check if demographics features in the data
demographic_features = list(set([
    'AGE',
    'EMPLOY',
    'DIVISION',
    'LIVARAG',
    'PRIMINC',
    'HLTHINS',
    'RACE',
    'PSYPROB',
    'DETNLF',
    'VET',
    'EDUC',
]).intersection(set(df.columns)))

In [9]:
# Model training
from sklearn.metrics import classification_report
def train_models_and_evaluate(X_train, y_train, X_test, y_test,
                              models=models_to_train,
                              ):
    """
    Trains models, returns evaulation metrics and also saves the results
    to a csv file in the local directory.
    Returns a list of train model instances.
    """
    df_metrics = pd.DataFrame(columns=['model', 'accuracy', 'precision', 'recall', 'f1'])
    trained_models = []
    X_train = X_train
    X_test = X_test
    for m in models:
        model = m()
        print(f"Fitting {model}...")
        model.fit(X_train, y_train)
        print(f"Predicting for {model}...")
        y_pred = model.predict(X_test)
        target_names=['0','1']
        print(classification_report(y_test, y_pred, target_names=target_names))
        # accuracy = accuracy_score(y_test, y_pred)
        # precision = precision_score(y_pred, y_test)
        # f1 = f1_score(y_pred, y_test)
        # recall = recall_score(y_pred, y_test)
        # print(str(m), accuracy, precision, recall, f1)
        df_metrics.loc[len(df_metrics.index)] = [str(m), 'accuracy', 'precision', 'recall', 'f1']
        trained_models.append(model)
    df_metrics.to_csv("models.csv", index=False)
    return trained_models

In [10]:
ms = train_models_and_evaluate(X_train, y_train, X_test, y_test)

Fitting RandomForestClassifier()...
Predicting for RandomForestClassifier()...
              precision    recall  f1-score   support

           0       0.83      0.91      0.87    360825
           1       0.73      0.57      0.64    155926

    accuracy                           0.81    516751
   macro avg       0.78      0.74      0.75    516751
weighted avg       0.80      0.81      0.80    516751

Fitting XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=Non

In [11]:
# Unfariness metrics


import sklearn.metrics as skm
import functools
from fairlearn.metrics import equalized_odds_difference, equalized_odds_ratio

fbeta_06 = functools.partial(skm.fbeta_score, beta=0.6)
fbeta_04 = functools.partial(skm.fbeta_score, beta=0.4)
fbeta_05 = functools.partial(skm.fbeta_score, beta=0.5)
def f1(y_true,y_pred):
    return skm.f1_score(y_true,y_pred)

def auc(y_true,y_pred):
    return skm.roc_auc_score(y_true,y_pred)

metric_fns = {'selection_rate': selection_rate,
              'fbeta_06': fbeta_06,
              'fbeta_04': fbeta_04,
              'fbeta_05': fbeta_05,
              'count': count,
              'f1' : f1,
              'auc' : auc}


def unfairness_metrics_evaluation(models, X_test, y_test, prefix="unmitigated"):
    """
    Saves the unfairness metrics for all models in a csv file.
    Returns a list of dictionaries with key as model object and values
    as a dictionary of unfairness metrics and their values.
    """
    df_unfairness_metrics = pd.DataFrame(columns=['categories',
                                                  'demographic_feature',
                                                  'model',
                                                  'selection_rate',
                                                  'fbeta_06',
                                                  'fbeta_04',
                                                  'fbeta_05',
                                                  'count',
                                                 'f1',
                                                 'auc'])
    df_equalized_odds = pd.DataFrame(columns=['demographic_feature',
                                              'model',
                                              'equalized_odds_difference',
                                              'equalized_odds_ratio',
                                              'demographic_parity_ratio',
                                              'demographic_parity_difference',
                                              'false_positive_rate_difference',
                                                    'false_negative_rate_difference',
                                                    'true_positive_rate_difference',
                                                     'true_negative_rate_difference',
                                                    ])

    df_overall_model_metrics = pd.DataFrame(columns=['model',
                                                    'false_positive_rate_difference',
                                                    'false_negative_rate_difference',
                                                    'true_positive_rate_difference',
                                                     'true_negative_rate_difference',
                                                    'overall_balance_error_rate'])
    for m in models:
        y_pred = m.predict(X_test)
        metrics = []
        for d in demographic_features:
            print("Model:", m)
            print("Demographic feature", d)
            results = MetricFrame(metrics=metric_fns,
                                y_true=y_test,
                                y_pred=y_pred,
                                sensitive_features=X_test[[d]])
            results = results.by_group.reset_index()
            print('result')
            results.rename({str(d): 'categories'}, inplace=True)
            results['model'] = str(m)
            print("result2")
            results['demographic_feature'] = str(d)
            print(results)
            df_unfairness_metrics = df_unfairness_metrics.append(results, ignore_index=True)
            print('result1')
            equalized_odds_diff = equalized_odds_difference(y_test, y_pred, sensitive_features=X_test[[d]])
            equalized_odds_r = equalized_odds_ratio(y_test, y_pred, sensitive_features=X_test[[d]])
            demographic_parity_r = demographic_parity_ratio(y_test, y_pred, sensitive_features=X_test[[d]])
            demographic_parity_diff = demographic_parity_difference(y_test, y_pred, sensitive_features=X_test[[d]])

            print("result4")
            false_positive_rate_diff = false_positive_rate_difference(y_test, y_pred, sensitive_features=X_test[[d]])
            false_negative_rate_diff = false_negative_rate_difference(y_test, y_pred, sensitive_features=X_test[[d]])
            true_positive_rate_diff = true_positive_rate_difference(y_test, y_pred, sensitive_features=X_test[[d]])
            true_negative_rate_diff = true_negative_rate_difference(y_test, y_pred, sensitive_features=X_test[[d]])
            # overall_balance_error = overall_balance_error_rate(y_test, y_pred, sensitive_features=X_test[d])
            print("result5")
            df_equalized_odds = df_equalized_odds.append(pd.DataFrame([[str(d), str(m),
                                                   equalized_odds_difference,
                                                   equalized_odds_ratio,
                                                   demographic_parity_ratio,
                                                   demographic_parity_difference,
                                                   false_positive_rate_diff,
                                                      false_negative_rate_diff,
                                                      true_positive_rate_diff,
                                                      true_negative_rate_diff]]),
                                     ignore_index=True)
            print("result6")

            # To do:
            # 1. Add true positive rate difference
            # 2. Add false positive rate difference

    df_equalized_odds.to_csv(f"{prefix}_Equalized_odds.csv", index=False)
    # df_overall_model_metrics.to_csv("Overall_metrics.csv", index=False)
    df_unfairness_metrics.to_csv(f"{prefix}_Unfairness_metrics.csv", index=False)
    return

In [12]:
unfairness_metrics_evaluation(ms, X_test, y_test)

Model: RandomForestClassifier()
Demographic feature EDUC
result
result2
   EDUC  selection_rate  fbeta_06  fbeta_04  fbeta_05     count        f1  \
0    -9        0.115785  0.581930  0.626909  0.604050   52390.0  0.513544   
1     1        0.257964  0.700006  0.721130  0.710630   26713.0  0.663910   
2     2        0.274005  0.686458  0.706517  0.696552   96761.0  0.652095   
3     3        0.237817  0.681548  0.704701  0.693172  221162.0  0.642376   
4     4        0.244249  0.683724  0.705933  0.694882   91591.0  0.646003   
5     5        0.242482  0.672411  0.696938  0.684710   28134.0  0.631183   

        auc                     model demographic_feature  
0  0.682832  RandomForestClassifier()                EDUC  
1  0.750817  RandomForestClassifier()                EDUC  
2  0.738659  RandomForestClassifier()                EDUC  
3  0.740401  RandomForestClassifier()                EDUC  
4  0.741698  RandomForestClassifier()                EDUC  
5  0.730661  RandomForestCla

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
demographic_features

['EDUC',
 'DETNLF',
 'LIVARAG',
 'VET',
 'PRIMINC',
 'DIVISION',
 'HLTHINS',
 'RACE',
 'PSYPROB',
 'AGE',
 'EMPLOY']

In [15]:
X=df.drop(columns = ["L/S"])
y = df['L/S']
#A = df['ETHNIC']
B = df['HLTHINS']
C = df['LIVARAG']
D = df['RACE']
E = df['PSYPROB']
F = df['PRIMINC']
G = df['AGE']
H = df['EMPLOY']
I = df['DETNLF']
#J = df['MARSTAT']
#K = df['GENDER']
L = df['VET']
M = df['EDUC']
N = df['DIVISION']
X_train,X_test,y_train,y_test,B_train,B_test,C_train,C_test,D_train,D_test,E_train,E_test,F_train,F_test,G_train,G_test,H_train,H_test,I_train,I_test,L_train,L_test,M_train,M_test,N_train,N_test= train_test_split(df.drop(columns = ["L/S"]),
                                                                     df['L/S'],
                                                                     B,C,D,E,F,G,H,I,L,M,N,
                                                                     stratify=df['L/S'],
                                                                     test_size=0.3)

In [16]:
# # Fairlearn algorithms and utils
# from fairlearn.postprocessing import ThresholdOptimizer
# from fairlearn.reductions import GridSearch, EqualizedOdds

# Metrics
from fairlearn.metrics import (
    MetricFrame,
    selection_rate, demographic_parity_difference, demographic_parity_ratio,
    false_positive_rate, false_negative_rate,
    false_positive_rate_difference, false_negative_rate_difference,
    equalized_odds_difference)
from sklearn.metrics import balanced_accuracy_score, roc_auc_score

In [17]:
# Helper functions
def get_metrics_df(models_dict, y_true, group):
    metrics_dict = {
        "Overall selection rate": (
            lambda x: selection_rate(y_true, x), True),
        "Demographic parity difference": (
            lambda x: demographic_parity_difference(y_true, x, sensitive_features=group), True),
        "Demographic parity ratio": (
            lambda x: demographic_parity_ratio(y_true, x, sensitive_features=group), True),
        "------": (lambda x: "", True),
        "Overall balanced error rate": (
            lambda x: 1-balanced_accuracy_score(y_true, x), True),
        "Balanced error rate difference": (
            lambda x: MetricFrame(metrics=balanced_accuracy_score, y_true=y_true, y_pred=x, sensitive_features=group).difference(method='between_groups'), True),
        " ------": (lambda x: "", True),
        "False positive rate difference": (
            lambda x: false_positive_rate_difference(y_true, x, sensitive_features=group), True),
         " ------": (lambda x: "", True),

        "False negative rate difference": (
            lambda x: false_negative_rate_difference(y_true, x, sensitive_features=group), True),
        "Equalized odds difference": (
            lambda x: equalized_odds_difference(y_true, x, sensitive_features=group), True),
        "  ------": (lambda x: "", True),
        "Overall AUC": (
            lambda x: roc_auc_score(y_true, x), False),
        "AUC difference": (
            lambda x: MetricFrame(metrics=roc_auc_score, y_true=y_true, y_pred=x, sensitive_features=group).difference(method='between_groups'), False),
        "F1 Score:" : (
            lambda x: f1_score(y_true,x), True)
    }
    df_dict = {}
    for metric_name, (metric_func, use_preds) in metrics_dict.items():
        df_dict[metric_name] = [metric_func(preds) if use_preds else metric_func(scores)
                                for model_name, (preds, scores) in models_dict.items()]
    return pd.DataFrame.from_dict(df_dict, orient="index", columns=models_dict.keys())

In [18]:
# Model Training with RF
rnd = RandomForestClassifier(n_estimators=100, max_depth =1, random_state=42).fit(X_train,y_train)
rnd.score(X_test, y_test)
y_pred = pd.DataFrame(rnd.predict(X_test),columns=["Passed"])
target_names=['0','1']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

           0       0.70      1.00      0.82    360583
           1       0.00      0.00      0.00    156168

    accuracy                           0.70    516751
   macro avg       0.35      0.50      0.41    516751
weighted avg       0.49      0.70      0.57    516751



In [19]:
test_scores = rnd.predict_proba(X_test)[:, 1]
test_preds = (test_scores >= np.mean(y_train)) * 1
# Train AUC
roc_auc_score(y_train, rnd.predict_proba(X_train)[:, 1])

0.6976979321286341

In [20]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=B_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
HLTHINS,,
-9,0.382470,0.294951
1,0.483193,0.270311
2,0.275105,0.490689
3,0.395107,0.365266
4,0.493116,0.245318


In [21]:
# HLTHINS Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, B_test)

,Unmitigated
Overall selection rate,0.463186
Demographic parity difference,0.262208
Demographic parity ratio,0.557229
------,
Overall balanced error rate,0.348938
Balanced error rate difference,0.044186
------,
False positive rate difference,0.218011
False negative rate difference,0.245371
Equalized odds difference,0.245371


In [22]:
#Livgrad
# HLTHINS Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, B_test)

,Unmitigated
Overall selection rate,0.463186
Demographic parity difference,0.262208
Demographic parity ratio,0.557229
------,
Overall balanced error rate,0.348938
Balanced error rate difference,0.044186
------,
False positive rate difference,0.218011
False negative rate difference,0.245371
Equalized odds difference,0.245371


In [23]:
# LIVARAG Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, C_test)

,Unmitigated
Overall selection rate,0.463186
Demographic parity difference,0.50975
Demographic parity ratio,0.219362
------,
Overall balanced error rate,0.348938
Balanced error rate difference,0.081007
------,
False positive rate difference,0.449858
False negative rate difference,0.519466
Equalized odds difference,0.519466


In [24]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=D_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
RACE,,
-9,0.186198,0.563733
1,0.316619,0.373418
2,0.378597,0.300091
3,0.458333,0.472222
4,0.371567,0.333108
5,0.370600,0.328904
6,0.488994,0.215008
7,0.441901,0.251013
8,0.483496,0.307760


In [25]:
# RACE Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, D_test)

,Unmitigated
Overall selection rate,0.463186
Demographic parity difference,0.391825
Demographic parity ratio,0.3741
------,
Overall balanced error rate,0.348938
Balanced error rate difference,0.125934
------,
False positive rate difference,0.359106
False negative rate difference,0.348724
Equalized odds difference,0.359106


In [26]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=E_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
PSYPROB,,
-9,0.330174,0.233139
1,0.369200,0.380323
2,0.385872,0.308182


In [27]:
# PSYPROB Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, E_test)

,Unmitigated
Overall selection rate,0.463186
Demographic parity difference,0.039564
Demographic parity ratio,0.917896
------,
Overall balanced error rate,0.348938
Balanced error rate difference,0.093105
------,
False positive rate difference,0.055698
False negative rate difference,0.147184
Equalized odds difference,0.147184


In [28]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=F_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
PRIMINC,,
-9,0.320718,0.387496
1,0.520423,0.197883
2,0.370172,0.341168
3,0.377636,0.394115
4,0.380748,0.297113
5,0.349236,0.340079


In [29]:
# PRIMINC Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, F_test)

,Unmitigated
Overall selection rate,0.463186
Demographic parity difference,0.214408
Demographic parity ratio,0.654532
------,
Overall balanced error rate,0.348938
Balanced error rate difference,0.046945
------,
False positive rate difference,0.199706
False negative rate difference,0.196232
Equalized odds difference,0.199706


In [30]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=G_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
AGE,,
1,0.706497,0.154734
2,0.742490,0.148197
3,0.555123,0.215065
4,0.471705,0.264022
5,0.396647,0.308926
6,0.378071,0.327221
7,0.365018,0.327513
8,0.344727,0.345199
9,0.321387,0.376130


In [31]:
# AGE Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, G_test)

,Unmitigated
Overall selection rate,0.463186
Demographic parity difference,0.421163
Demographic parity ratio,0.469328
------,
Overall balanced error rate,0.348938
Balanced error rate difference,0.108038
------,
False positive rate difference,0.456085
False negative rate difference,0.254343
Equalized odds difference,0.456085


In [32]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=H_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
EMPLOY,,
-9,0.078434,0.656035
1,0.549495,0.203203
2,0.516560,0.252040
3,0.364407,0.341548
4,0.360493,0.331268


In [33]:
# EMPLOY Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, H_test)

,Unmitigated
Overall selection rate,0.463186
Demographic parity difference,0.498488
Demographic parity ratio,0.221192
------,
Overall balanced error rate,0.348938
Balanced error rate difference,0.038419
------,
False positive rate difference,0.471061
False negative rate difference,0.452832
Equalized odds difference,0.471061


In [34]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=I_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
DETNLF,,
-9,0.391406,0.311988
1,0.475071,0.294227
2,0.630726,0.192646
3,0.372781,0.392307
4,0.717691,0.141026
5,0.238461,0.441840


In [35]:
# DETNLF Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, I_test)

,Unmitigated
Overall selection rate,0.463186
Demographic parity difference,0.464763
Demographic parity ratio,0.403976
------,
Overall balanced error rate,0.348938
Balanced error rate difference,0.089208
------,
False positive rate difference,0.47923
False negative rate difference,0.300814
Equalized odds difference,0.47923


In [36]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=L_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
VET,,
-9,0.153454,0.526831
1,0.415035,0.302879
2,0.397785,0.306981


In [37]:
# VET Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, L_test)

,Unmitigated
Overall selection rate,0.463186
Demographic parity difference,0.271097
Demographic parity ratio,0.465714
------,
Overall balanced error rate,0.348938
Balanced error rate difference,0.018815
------,
False positive rate difference,0.261581
False negative rate difference,0.223952
Equalized odds difference,0.261581


In [38]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=M_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
EDUC,,
-9,0.112628,0.729690
1,0.374337,0.315685
2,0.461205,0.271811
3,0.395969,0.303413
4,0.390555,0.306956
5,0.394328,0.308802


In [39]:
# EDUC Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, M_test)

,Unmitigated
Overall selection rate,0.463186
Demographic parity difference,0.409716
Demographic parity ratio,0.258658
------,
Overall balanced error rate,0.348938
Balanced error rate difference,0.076148
------,
False positive rate difference,0.348577
False negative rate difference,0.457879
Equalized odds difference,0.457879


In [40]:
mf = MetricFrame({
    'FPR': false_positive_rate,
    'FNR': false_negative_rate},
    y_test, test_preds, sensitive_features=N_test)
pd.set_option('display.max_rows', None)
mf.by_group

,FPR,FNR
DIVISION,,
0,0.360465,0.548571
1,0.260480,0.515379
2,0.356248,0.281077
3,0.402467,0.350025
4,0.633104,0.208027
5,0.275329,0.394481
6,0.514692,0.290936
7,0.427180,0.342688
8,0.203565,0.452284


In [90]:
# DIVISION Metrics
models_dict = {"Unmitigated": (test_preds, test_scores)}
get_metrics_df(models_dict, y_test, N_test)

,Unmitigated
Overall selection rate,0.463186
Demographic parity difference,0.484537
Demographic parity ratio,0.364195
------,
Overall balanced error rate,0.348938
Balanced error rate difference,0.135856
------,
False positive rate difference,0.506108
False negative rate difference,0.378802
Equalized odds difference,0.506108


In [89]:
LIST=['DIVISION',
 'RACE',
 'AGE',
 'EDUC',
 'PRIMINC',
 'HLTHINS',
 'LIVARAG',
 'VET',
 'PSYPROB',
 'DETNLF',
 'EMPLOY']

In [84]:
from fairlearn.reductions import GridSearch, DemographicParity
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from fairlearn.reductions import GridSearch, DemographicParity, ExponentiatedGradient
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [94]:
#Model Training 
clf = XGBClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42).fit(X_train, y_train)
clf.score(X_test, y_test)
y_pred = pd.DataFrame(clf.predict(X_test),columns=["Passed"])
target_names=['0','1']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

           0       0.77      0.89      0.82    360583
           1       0.60      0.38      0.46    156168

    accuracy                           0.74    516751
   macro avg       0.68      0.63      0.64    516751
weighted avg       0.72      0.74      0.72    516751



In [ ]:
#A = df['ETHNIC']
B = df['HLTHINS']
C = df['LIVARAG']
#D = df['RACE']
#E = df['PSYPROB']
#F = df['PRIMINC']
#G = df['AGE']
#H = df['EMPLOY']
#I = df['DETNLF']
#J = df['MARSTAT']
#K = df['GENDER']
L = df['VET']
M = df['EDUC']
N = df['DIVISION']

In [ ]:
#HLTHINS-B

In [96]:
postprocess_est = ThresholdOptimizer(
    estimator=clf,
    constraints="equalized_odds",
    prefit=True)
# Get index of positive label
# Get randomly index of negative label but the number of indices has to be the same as the positive one
# Combine negative and postive indices
balanced_idx1 = X_train[y_train==1].index
pp_train_idx = balanced_idx1.union(y_train[y_train==0].sample(n=balanced_idx1.size,replace= True, random_state=1234).index)

# Get a new training dataset based on created indices
X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
B_train_balanced = B_train.loc[pp_train_idx]

In [97]:
postprocess_est

ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=1.0, max_bin=None,
                                           max_cat_thre

In [100]:
# Model Training
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=B_train_balanced)

C:\Users\janni\anaconda3\lib\site-packages\fairlearn\postprocessing\_threshold_optimizer.py:292: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  warn(


ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=1.0, max_bin=None,
                                           max_cat_thre

In [101]:
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=B_test)

In [102]:
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
get_metrics_df(models_dict, y_test, B_test)

,Unmitigated,ThresholdOptimizer
Overall selection rate,0.463186,0.08387
Demographic parity difference,0.50975,0.014918
Demographic parity ratio,0.219362,0.83546
------,,
Overall balanced error rate,0.348938,0.461517
Balanced error rate difference,0.081007,0.00537
------,,
False positive rate difference,0.449858,0.002695
False negative rate difference,0.519466,0.008347
Equalized odds difference,0.519466,0.008347


In [ ]:
#LIVARAG-C

In [103]:
postprocess_est = ThresholdOptimizer(
    estimator=clf,
    constraints="equalized_odds",
    prefit=True)
# Get index of positive label
# Get randomly index of negative label but the number of indices has to be the same as the positive one
# Combine negative and postive indices
balanced_idx1 = X_train[y_train==1].index
pp_train_idx = balanced_idx1.union(y_train[y_train==0].sample(n=balanced_idx1.size,replace= True, random_state=1234).index)

# Get a new training dataset based on created indices
X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
C_train_balanced = C_train.loc[pp_train_idx]

In [104]:
postprocess_est

ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=1.0, max_bin=None,
                                           max_cat_thre

In [105]:
# Model Training
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=C_train_balanced)

C:\Users\janni\anaconda3\lib\site-packages\fairlearn\postprocessing\_threshold_optimizer.py:292: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  warn(


ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=1.0, max_bin=None,
                                           max_cat_thre

In [106]:
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=C_test)

In [107]:
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
get_metrics_df(models_dict, y_test, C_test)

,Unmitigated,ThresholdOptimizer
Overall selection rate,0.463186,0.124147
Demographic parity difference,0.391825,0.045186
Demographic parity ratio,0.3741,0.715973
------,,
Overall balanced error rate,0.348938,0.433959
Balanced error rate difference,0.125934,0.02041
------,,
False positive rate difference,0.359106,0.038652
False negative rate difference,0.348724,0.063504
Equalized odds difference,0.359106,0.063504


In [ ]:
#D RACE

In [108]:
postprocess_est = ThresholdOptimizer(
    estimator=clf,
    constraints="equalized_odds",
    prefit=True)
# Get index of positive label
# Get randomly index of negative label but the number of indices has to be the same as the positive one
# Combine negative and postive indices
balanced_idx1 = X_train[y_train==1].index
pp_train_idx = balanced_idx1.union(y_train[y_train==0].sample(n=balanced_idx1.size,replace= True, random_state=1234).index)

# Get a new training dataset based on created indices
X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
D_train_balanced = D_train.loc[pp_train_idx]

In [109]:
postprocess_est

ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=1.0, max_bin=None,
                                           max_cat_thre

In [110]:
# Model Training
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=D_train_balanced)

C:\Users\janni\anaconda3\lib\site-packages\fairlearn\postprocessing\_threshold_optimizer.py:292: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  warn(


ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=1.0, max_bin=None,
                                           max_cat_thre

In [111]:
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=D_test)

In [112]:
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
get_metrics_df(models_dict, y_test, D_test)

,Unmitigated,ThresholdOptimizer
Overall selection rate,0.463186,0.123214
Demographic parity difference,0.391825,0.101719
Demographic parity ratio,0.3741,0.520465
------,,
Overall balanced error rate,0.348938,0.433731
Balanced error rate difference,0.125934,0.046107
------,,
False positive rate difference,0.359106,0.084617
False negative rate difference,0.348724,0.158579
Equalized odds difference,0.359106,0.158579


In [ ]:
#PSYPROB-E

In [113]:
postprocess_est = ThresholdOptimizer(
    estimator=clf,
    constraints="equalized_odds",
    prefit=True)
# Get index of positive label
# Get randomly index of negative label but the number of indices has to be the same as the positive one
# Combine negative and postive indices
balanced_idx1 = X_train[y_train==1].index
pp_train_idx = balanced_idx1.union(y_train[y_train==0].sample(n=balanced_idx1.size,replace= True, random_state=1234).index)

# Get a new training dataset based on created indices
X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
E_train_balanced = E_train.loc[pp_train_idx]

In [114]:
postprocess_est

ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=1.0, max_bin=None,
                                           max_cat_thre

In [115]:
# Model Training
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=E_train_balanced)

C:\Users\janni\anaconda3\lib\site-packages\fairlearn\postprocessing\_threshold_optimizer.py:292: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  warn(


ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=1.0, max_bin=None,
                                           max_cat_thre

In [116]:
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=E_test)

In [117]:
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
get_metrics_df(models_dict, y_test, E_test)

,Unmitigated,ThresholdOptimizer
Overall selection rate,0.463186,0.183957
Demographic parity difference,0.039564,0.005642
Demographic parity ratio,0.917896,0.969796
------,,
Overall balanced error rate,0.348938,0.381925
Balanced error rate difference,0.093105,0.000806
------,,
False positive rate difference,0.055698,0.001117
False negative rate difference,0.147184,0.001447
Equalized odds difference,0.147184,0.001447


In [118]:
#PRIMINC-F

In [119]:
postprocess_est = ThresholdOptimizer(
    estimator=clf,
    constraints="equalized_odds",
    prefit=True)
# Get index of positive label
# Get randomly index of negative label but the number of indices has to be the same as the positive one
# Combine negative and postive indices
balanced_idx1 = X_train[y_train==1].index
pp_train_idx = balanced_idx1.union(y_train[y_train==0].sample(n=balanced_idx1.size,replace= True, random_state=1234).index)

# Get a new training dataset based on created indices
X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
F_train_balanced = F_train.loc[pp_train_idx]

In [120]:
postprocess_est

ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=1.0, max_bin=None,
                                           max_cat_thre

In [121]:
# Model Training
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=F_train_balanced)

C:\Users\janni\anaconda3\lib\site-packages\fairlearn\postprocessing\_threshold_optimizer.py:292: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  warn(


ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=1.0, max_bin=None,
                                           max_cat_thre

In [122]:
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=F_test)

In [123]:
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
get_metrics_df(models_dict, y_test, F_test)

,Unmitigated,ThresholdOptimizer
Overall selection rate,0.463186,0.206751
Demographic parity difference,0.214408,0.016163
Demographic parity ratio,0.654532,0.925657
------,,
Overall balanced error rate,0.348938,0.395019
Balanced error rate difference,0.046945,0.003578
------,,
False positive rate difference,0.199706,0.003675
False negative rate difference,0.196232,0.008642
Equalized odds difference,0.199706,0.008642


In [124]:
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=F_test)

In [ ]:
#AGE-G

In [125]:
postprocess_est = ThresholdOptimizer(
    estimator=clf,
    constraints="equalized_odds",
    prefit=True)
# Get index of positive label
# Get randomly index of negative label but the number of indices has to be the same as the positive one
# Combine negative and postive indices
balanced_idx1 = X_train[y_train==1].index
pp_train_idx = balanced_idx1.union(y_train[y_train==0].sample(n=balanced_idx1.size,replace= True, random_state=1234).index)

# Get a new training dataset based on created indices
X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
G_train_balanced = G_train.loc[pp_train_idx]

In [126]:
postprocess_est

ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=1.0, max_bin=None,
                                           max_cat_thre

In [127]:
# Model Training
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=G_train_balanced)

C:\Users\janni\anaconda3\lib\site-packages\fairlearn\postprocessing\_threshold_optimizer.py:292: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  warn(


ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=1.0, max_bin=None,
                                           max_cat_thre

In [128]:
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=G_test)

In [129]:
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
get_metrics_df(models_dict, y_test, G_test)

,Unmitigated,ThresholdOptimizer
Overall selection rate,0.463186,0.428522
Demographic parity difference,0.421163,0.036889
Demographic parity ratio,0.469328,0.919675
------,,
Overall balanced error rate,0.348938,0.420803
Balanced error rate difference,0.108038,0.006137
------,,
False positive rate difference,0.456085,0.011931
False negative rate difference,0.254343,0.015768
Equalized odds difference,0.456085,0.015768


In [ ]:
#EMPLOY-H

In [ ]:
postprocess_est = ThresholdOptimizer(
    estimator=clf,
    constraints="equalized_odds",
    prefit=True)
# Get index of positive label
# Get randomly index of negative label but the number of indices has to be the same as the positive one
# Combine negative and postive indices
balanced_idx1 = X_train[y_train==1].index
pp_train_idx = balanced_idx1.union(y_train[y_train==0].sample(n=balanced_idx1.size,replace= True, random_state=1234).index)

# Get a new training dataset based on created indices
X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
H_train_balanced = H_train.loc[pp_train_idx]

In [ ]:
postprocess_est

In [ ]:
# Model Training
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=H_train_balanced)

In [130]:
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=D_test)

In [131]:
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
get_metrics_df(models_dict, y_test, H_test)

,Unmitigated,ThresholdOptimizer
Overall selection rate,0.463186,0.403204
Demographic parity difference,0.498488,0.199208
Demographic parity ratio,0.221192,0.577147
------,,
Overall balanced error rate,0.348938,0.416995
Balanced error rate difference,0.038419,0.019356
------,,
False positive rate difference,0.471061,0.185374
False negative rate difference,0.452832,0.1562
Equalized odds difference,0.471061,0.185374


In [ ]:
#DETNLF-I

In [132]:
postprocess_est = ThresholdOptimizer(
    estimator=clf,
    constraints="equalized_odds",
    prefit=True)
# Get index of positive label
# Get randomly index of negative label but the number of indices has to be the same as the positive one
# Combine negative and postive indices
balanced_idx1 = X_train[y_train==1].index
pp_train_idx = balanced_idx1.union(y_train[y_train==0].sample(n=balanced_idx1.size,replace= True, random_state=1234).index)

# Get a new training dataset based on created indices
X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
I_train_balanced = I_train.loc[pp_train_idx]

In [133]:
postprocess_est

ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=1.0, max_bin=None,
                                           max_cat_thre

In [134]:
# Model Training
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=I_train_balanced)

C:\Users\janni\anaconda3\lib\site-packages\fairlearn\postprocessing\_threshold_optimizer.py:292: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  warn(


ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=1.0, max_bin=None,
                                           max_cat_thre

In [135]:
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=I_test)

In [136]:
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
get_metrics_df(models_dict, y_test, I_test)

,Unmitigated,ThresholdOptimizer
Overall selection rate,0.463186,0.259659
Demographic parity difference,0.464763,0.03573
Demographic parity ratio,0.403976,0.874953
------,,
Overall balanced error rate,0.348938,0.419165
Balanced error rate difference,0.089208,0.008509
------,,
False positive rate difference,0.47923,0.011081
False negative rate difference,0.300814,0.020463
Equalized odds difference,0.47923,0.020463


In [ ]:
# VET

In [150]:
postprocess_est = ThresholdOptimizer(
    estimator=clf,
    constraints="equalized_odds",
    prefit=True)
# Get index of positive label
# Get randomly index of negative label but the number of indices has to be the same as the positive one
# Combine negative and postive indices
balanced_idx1 = X_train[y_train==1].index
pp_train_idx = balanced_idx1.union(y_train[y_train==0].sample(n=balanced_idx1.size,replace= True, random_state=1234).index)

# Get a new training dataset based on created indices
X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
L_train_balanced = L_train.loc[pp_train_idx]

In [151]:
postprocess_est

ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=1.0, max_bin=None,
                                           max_cat_thre

In [152]:
# Model Training
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=L_train_balanced)

C:\Users\janni\anaconda3\lib\site-packages\fairlearn\postprocessing\_threshold_optimizer.py:292: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  warn(


ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=1.0, max_bin=None,
                                           max_cat_thre

In [153]:
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=D_test)

In [154]:
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
get_metrics_df(models_dict, y_test, L_test)

,Unmitigated,ThresholdOptimizer
Overall selection rate,0.463186,0.007201
Demographic parity difference,0.271097,0.007213
Demographic parity ratio,0.465714,0.430557
------,,
Overall balanced error rate,0.348938,0.495666
Balanced error rate difference,0.018815,0.000713
------,,
False positive rate difference,0.261581,0.007467
False negative rate difference,0.223952,0.008449
Equalized odds difference,0.261581,0.008449


In [ ]:
#M EDUCATION

In [145]:
postprocess_est = ThresholdOptimizer(
    estimator=clf,
    constraints="equalized_odds",
    prefit=True)
# Get index of positive label
# Get randomly index of negative label but the number of indices has to be the same as the positive one
# Combine negative and postive indices
balanced_idx1 = X_train[y_train==1].index
pp_train_idx = balanced_idx1.union(y_train[y_train==0].sample(n=balanced_idx1.size,replace= True, random_state=1234).index)

# Get a new training dataset based on created indices
X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
M_train_balanced = M_train.loc[pp_train_idx]

In [146]:
postprocess_est

ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=1.0, max_bin=None,
                                           max_cat_thre

In [147]:
# Model Training
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=M_train_balanced)

C:\Users\janni\anaconda3\lib\site-packages\fairlearn\postprocessing\_threshold_optimizer.py:292: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  warn(


ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=1.0, max_bin=None,
                                           max_cat_thre

In [148]:
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=M_test)

In [149]:
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
get_metrics_df(models_dict, y_test, M_test)

,Unmitigated,ThresholdOptimizer
Overall selection rate,0.463186,0.066754
Demographic parity difference,0.409716,0.010128
Demographic parity ratio,0.258658,0.855352
------,,
Overall balanced error rate,0.348938,0.459865
Balanced error rate difference,0.076148,0.004186
------,,
False positive rate difference,0.348577,0.00416
False negative rate difference,0.457879,0.007445
Equalized odds difference,0.457879,0.007445


In [ ]:
#N-DIVISION

In [139]:
postprocess_est = ThresholdOptimizer(
    estimator=clf,
    constraints="equalized_odds",
    prefit=True)
# Get index of positive label
# Get randomly index of negative label but the number of indices has to be the same as the positive one
# Combine negative and postive indices
balanced_idx1 = X_train[y_train==1].index
pp_train_idx = balanced_idx1.union(y_train[y_train==0].sample(n=balanced_idx1.size,replace= True, random_state=1234).index)

# Get a new training dataset based on created indices
X_train_balanced = X_train.loc[pp_train_idx, :]
y_train_balanced = y_train.loc[pp_train_idx]
N_train_balanced = N_train.loc[pp_train_idx]

In [140]:
postprocess_est

ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=1.0, max_bin=None,
                                           max_cat_thre

In [141]:
# Model Training
postprocess_est.fit(X_train_balanced, y_train_balanced, sensitive_features=N_train_balanced)

C:\Users\janni\anaconda3\lib\site-packages\fairlearn\postprocessing\_threshold_optimizer.py:292: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  warn(


ThresholdOptimizer(constraints='equalized_odds',
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=1.0, max_bin=None,
                                           max_cat_thre

In [142]:
postprocess_preds = postprocess_est.predict(X_test, sensitive_features=N_test)

In [163]:
models_dict = {"Unmitigated": (test_preds, test_scores),
              "ThresholdOptimizer": (postprocess_preds, postprocess_preds)}
get_metrics_df(models_dict, y_test, N_test)

,Unmitigated,ThresholdOptimizer
Overall selection rate,0.463186,0.007201
Demographic parity difference,0.484537,0.027148
Demographic parity ratio,0.364195,0.020402
------,,
Overall balanced error rate,0.348938,0.495666
Balanced error rate difference,0.135856,0.018969
------,,
False positive rate difference,0.506108,0.01157
False negative rate difference,0.378802,0.049291
Equalized odds difference,0.506108,0.049291


In [167]:
#

NameError: name 'clf' is not defined